In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
# get webpage with all packages
r = requests.get('https://conda-forge.org/feedstocks/')

# check the status code returned
r.status_code

In [ ]:
# parse the response to get package names
soup = BeautifulSoup(r.content, "lxml")
table = soup.find("section", {"id": "feedstocks"})
links = [a['href'] for a in table.findAll('a')]

# inspect the links
links[:3]

In [ ]:
# grab just the package names from the links
packages = [line.split('/')[-1].replace('-feedstock', '') for line in links]

# inspect the packages
packages[:3]

In [ ]:
# iterate over packages and get the grab meta.yaml
meta = {}
for pkg in packages:
    meta[pkg] = {}
    url = 'https://raw.githubusercontent.com/conda-forge/{}-feedstock/master/recipe/meta.yaml'.format(pkg)
    r = requests.get(url)
    if r.status_code == 200:
        meta[pkg]['response'] = r.text
    else:
        print("Error with {}".format(pkg))

In [ ]:
meta

In [ ]:
# find where the maintainers line starts
simplelist=[]
for pkg in meta:

    if meta[pkg] == {}:
        continue
    
    else:
        text = meta[pkg]['response'].split('\n')
        for i, line in enumerate(text):
            if 'recipe-maintainers' in line:
                break

        # grab the text from the maintainers section
        maintainers = text[i+1: ]

        # clean up the maintainers text
        maintainers_clean = []
        for m in maintainers:
            line = m.strip()
            if line != '' and not line.startswith('#'):
                maintainers_clean.append(line.replace('-', '').replace(' ', ''))

        # store the maintainers in meta dict
        meta[pkg]['maintainers'] = maintainers_clean
        simplelist.append([meta[pkg]['response'].split('\n')[0],maintainers_clean])

In [ ]:
#print(simplelist)
import pandas as pd
df = pd.DataFrame(simplelist)
print(df)

df.to_csv('maintainers_331.csv')